In [1]:
import os
import csv
import random
import pydicom
import numpy as np
import pandas as pd
from skimage import io
from skimage import measure
from skimage.transform import resize

import tensorflow as tf
from tensorflow import keras
import glob
from tqdm import tqdm
import cv2
import SimpleITK as sitk
from skimage.util import random_noise
from matplotlib import pyplot as plt
import matplotlib.patches as patches
#图片种类
det_class_path = './input/stage_1_detailed_class_info.csv'
#训练集中bbox的位置信息
bbox_path = './input/stage_1_train_labels.csv'
#训练集dcm格式图片
dcm_train_dir = './input/stage_1_train_images/'
#测试集dcm格式图片
dcm_test_dir = './input/stage_1_test_images/'
#测试集png图片
png_test_dir='./input/stage_1_test_png/'
#训练集png图片
png_train_dir='./input/stage_1_train_png/'
#存储npy文件
train_npy='./input/stage_1_train_npy/'
#获取所有dcm图片
dcm_paths =glob.glob('./input/**/*.dcm',recursive=True)
train_labels =  pd.read_csv(bbox_path)
class_info = pd.read_csv(det_class_path)


/Users/aier02/programme/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [17]:
len(dcm_paths)

26684

In [16]:
train_labels[:3]
print(train_labels['patientId'][:3])
labels=train_labels[train_labels['patientId']=='0004cfab-14fd-4e49-80ba-63a80b6bddd6']
labels.dropna()

0    0004cfab-14fd-4e49-80ba-63a80b6bddd6
1    00313ee0-9eaa-42f4-b0ab-c148ed3241cd
2    00322d4d-1c29-4943-afc9-b6754be640eb
Name: patientId, dtype: object


,patientId,x,y,width,height,Target


In [2]:
dcm_paths =glob.glob('./input/**/*.dcm',recursive=True)
dcm_paths[2000:2100]

['./input/stage_1_train_images/bb56b84a-b675-490b-b172-39b10b95142d.dcm',
 './input/stage_1_train_images/b12523d2-5618-4e33-9ad3-d1e77a9c9fcf.dcm',
 './input/stage_1_train_images/fe539da9-f41c-404f-bf48-cc20a30a201b.dcm',
 './input/stage_1_train_images/66184e1e-d09c-4cfc-9ff7-5b892566f140.dcm',
 './input/stage_1_train_images/d9763aaf-7377-45e0-a112-2ad0f3bb73ef.dcm',
 './input/stage_1_train_images/ccc3fd5e-9bf2-4185-a408-14d01ae1866b.dcm',
 './input/stage_1_train_images/f743c21e-5876-460b-bbd6-2e2dcca41baa.dcm',
 './input/stage_1_train_images/7e299a8b-364b-4348-bd16-c43ece7c9d12.dcm',
 './input/stage_1_train_images/d1392a45-212f-466c-90d4-d66e54cdf3ee.dcm',
 './input/stage_1_train_images/c23444bf-fd79-4fee-81eb-521e9fce470e.dcm',
 './input/stage_1_train_images/3e5b62ec-4b34-4f62-bb84-7b67421cb37c.dcm',
 './input/stage_1_train_images/8e0a650f-5894-4a7f-b606-5c8edf968d0d.dcm',
 './input/stage_1_train_images/db39adba-a659-4575-b424-7446e9db5b01.dcm',
 './input/stage_1_train_images/fce5610

In [3]:
#将所有的dcm图像转为png格式，同时存储患有肺炎的图像的numpy数组和bbox的位置信息，以文件的格式保存
for path in tqdm(dcm_paths):
    dcm=sitk.ReadImage(path)
    img_array=sitk.GetArrayFromImage(dcm)[0]
    patientid=path.split('/')[-1][:-4]
    if 'test' in path:
        save_path = os.path.join(png_test_dir,f'{patientid}.png')
        cv2.imwrite(save_path,img_array)
    else:
        labels = train_labels[train_labels['patientId'] == patientid]
        classes = class_info[class_info['patientId'] == patientid]
        #dropna指去掉空值
        num_label = len(labels.dropna())
        if num_label == 0:
            '''
            class_label = classes.iloc[0]['class']
            if class_label == 'Normal':
                save_path = os.path.join(png_train_dir+'negative/normal/',f'{patientid}.png')
                cv2.imwrite(save_path,img_array)
            else:
                save_path = os.path.join(png_train_dir+'negative/abnormal/',f'{patientid}.png')
                cv2.imwrite(save_path,img_array)
            '''
            save_path = os.path.join(png_train_dir,f'{patientid}.png')
            cv2.imwrite(save_path,img_array)
        else:
            bboxes = []
            copy_img = img_array.copy()
            for n in range(num_label):
                label = labels.iloc[n]
                x = label['x'] #upper-left
                y = label['y'] #upper-left y
                width = label['width'] 
                height = label['height']
                #bbox coord : (upper-left y,upper-left x,bottom-right y,bottom-right x)
                bbox = [y,x,y+height,x+width]
                bboxes.append(bbox)
            bboxes = np.array(bboxes)
            #验证集稍后分裂
            save_path = os.path.join(train_npy,f'{patientid}.npy')
            np.save(save_path,{'image':img_array,'bboxes':bboxes})
            save_path=os.path.join(png_train_dir,f'{patientid}.png')
            cv2.imwrite(save_path,img_array)

  0%|          | 0/26684 [00:00<?, ?it/s]

  0%|          | 1/26684 [00:00<1:15:33,  5.89it/s]

  0%|          | 3/26684 [00:00<41:26, 10.73it/s]  

  0%|          | 6/26684 [00:00<30:15, 14.70it/s]

  0%|          | 9/26684 [00:00<25:41, 17.31it/s]

...

100%|█████████▉| 26672/26684 [15:06<00:00, 29.42it/s]

100%|█████████▉| 26676/26684 [15:06<00:00, 29.42it/s]

100%|█████████▉| 26679/26684 [15:06<00:00, 29.42it/s]

100%|█████████▉| 26683/26684 [15:06<00:00, 29.42it/s]

100%|██████████| 26684/26684 [15:07<00:00, 29.42it/s]

In [3]:
normal_paths =glob.glob('./input/stage_1_train_png/negative/*/*.png',recursive=True)
normal_id=[]
for path in tqdm(normal_paths):
    patientid=path.split('/')[-1][:-4]
    normal_id.append(patientid)
for patientid in tqdm(normal_id):
    dcm=sitk.ReadImage(os.path.join(dcm_train_dir,patientid+'.dcm'))
    img_array=sitk.GetArrayFromImage(dcm)[0]
    save_path = os.path.join('./input/normal_npy',f'{patientid}.npy')
    np.save(save_path,{'image':img_array,'bboxes':[]})

  0%|          | 0/20025 [00:00<?, ?it/s]

100%|██████████| 20025/20025 [00:00<00:00, 534630.19it/s]


  0%|          | 0/20025 [00:00<?, ?it/s]

  0%|          | 1/20025 [00:00<33:56,  9.83it/s]

  0%|          | 6/20025 [00:00<11:53, 28.06it/s]

...

100%|█████████▉| 20006/20025 [05:35<00:00, 59.58it/s]

100%|█████████▉| 20012/20025 [05:35<00:00, 59.56it/s]

100%|█████████▉| 20017/20025 [05:36<00:00, 59.55it/s]

100%|█████████▉| 20022/20025 [05:36<00:00, 59.54it/s]

100%|██████████| 20025/20025 [05:36<00:00, 59.54it/s]

In [4]:
file_test=[]
for path in tqdm(dcm_paths):
    dcm=sitk.ReadImage(path)
    img_array=sitk.GetArrayFromImage(dcm)[0]
    patientid=path.split('/')[-1][:-4]
    if 'train' in path:
        labels = train_labels[train_labels['patientId'] == patientid]
        classes = class_info[class_info['patientId'] == patientid]
        #dropna指去掉空值
        num_label = len(labels.dropna())
        bboxes=[]
        if num_label == 0:
            file_test.append(path)


  0%|          | 0/26684 [00:00<?, ?it/s]

  0%|          | 9/26684 [00:00<04:57, 89.56it/s]

  0%|          | 19/26684 [00:00<04:55, 90.39it/s]

  0%|          | 27/26684 [00:00<05:13, 84.92it/s]

  0%|          | 36/26684 [00:00<05:15, 84.58it/s]

...

100%|█████████▉| 26663/26684 [07:54<00:00, 56.14it/s]

100%|█████████▉| 26669/26684 [07:55<00:00, 56.14it/s]

100%|█████████▉| 26675/26684 [07:55<00:00, 56.14it/s]

100%|█████████▉| 26681/26684 [07:55<00:00, 56.14it/s]

100%|██████████| 26684/26684 [07:55<00:00, 56.14it/s]

In [6]:
print(len(file_test))
path=glob.glob('./input/stage_1_train_png/negative/*/*.png',recursive=True)
print(len(path))

20025
20025


In [3]:
#对一张dcm图像进行仿射变换
def rotate_img_bbox(img, bboxes, degree=-45, scale=1.):
    height, width = img.shape[:2]
    center = width//2,height//2 #中心坐标
    #得到以图像中心坐标的旋转角度为degrees的放射矩阵
    mat = cv2.getRotationMatrix2D(center,angle=degree,scale=scale) #affine matrix
    #对图像进行仿射变换
    out_image = cv2.warpAffine(img,mat,(width,height))
    #print(len(pneumonia_locations[pneumonia_id[3]]))
    #对bboxes进行仿射变化
    rot_bboxes = list()
    for bbox in bboxes:
        ymin = bbox[0]
        xmin = bbox[1]
        ymax = bbox[2]
        xmax = bbox[3]
        point1 = np.dot(mat, np.array([(xmin+xmax)/2, ymin, 1]))
        point2 = np.dot(mat, np.array([xmax, (ymin+ymax)/2, 1]))
        point3 = np.dot(mat, np.array([(xmin+xmax)/2, ymax, 1]))
        point4 = np.dot(mat, np.array([xmin, (ymin+ymax)/2, 1]))
        # 合并np.array
        concat = np.vstack((point1, point2, point3, point4))
        # 改变array类型
        concat = concat.astype(np.int32)
        # 得到旋转后的坐标
        rx, ry, rw, rh = cv2.boundingRect(concat)
        rx_min = rx
        ry_min = ry
        rx_max = rx+rw
        ry_max = ry+rh
        # 加入list中
        rot_bboxes.append([ry_min, rx_min, ry_max, rx_max])
    return out_image,rot_bboxes


In [4]:
# 对一张dcm图像进行平移
def shift_pic_bboxes(img, bboxes):
    '''
    参考:https://blog.csdn.net/sty945/article/details/79387054
    平移后的图片要包含所有的框
    输入:
        img:图像array
        bboxes:该图像包含的所有boundingboxs,一个list,每个元素为[y_min, x_min, y_max, x_max],要确保是数值
    输出:
        shift_img:平移后的图像array
        shift_bboxes:平移后的bounding box的坐标list
    '''
    #---------------------- 平移图像 ----------------------
    w = img.shape[1]
    h = img.shape[0]
    x_min = w   #裁剪后的包含所有目标框的最小的框
    x_max = 0
    y_min = h
    y_max = 0
    for bbox in bboxes:
        x_min = min(x_min, bbox[1])
        y_min = min(y_min, bbox[0])
        x_max = max(x_max, bbox[3])
        y_max = max(y_max, bbox[2])
    
    d_to_left = x_min           #包含所有目标框的最大左移动距离
    d_to_right = w - x_max      #包含所有目标框的最大右移动距离
    d_to_top = y_min            #包含所有目标框的最大上移动距离
    d_to_bottom = h - y_max     #包含所有目标框的最大下移动距离

    x = random.uniform(-(d_to_left-1) / 3, (d_to_right-1) / 3)
    y = random.uniform(-(d_to_top-1) / 3, (d_to_bottom-1) / 3)
    
    M = np.float32([[1, 0, x], [0, 1, y]])  #x为向左或右移动的像素值,正为向右负为向左; y为向上或者向下移动的像素值,正为向下负为向上
    shift_img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))

    #---------------------- 平移boundingbox ----------------------
    shift_bboxes = list()
    for bbox in bboxes:
        shift_bboxes.append([bbox[0]+y, bbox[1]+x, bbox[2]+y, bbox[3]+x])

    return shift_img, shift_bboxes

In [8]:
npy_path=glob.glob(train_npy+'*.npy')
print(len(npy_path))

5659


In [9]:
#对已有的训练集正样本进行扩展，每张图像分别进行一次评议，和一次仿射变换（进行过采样），注意将新的图
#像信息添加到stage_1_train_png，stage_1_train_npy,stage_1_train_labels.csv,stage_1
#_detailed_class_info.csv
npy_path=glob.glob(train_npy+'*.npy')
npy_num=len(npy_path)
with open(det_class_path, "a+",newline='') as class_csv:
    with open(bbox_path,"a+",newline='') as label_csv:
        writer_c=csv.writer(class_csv)
        writer_l=csv.writer(label_csv)
        for path in tqdm(npy_path):
            npy = np.load(path).item()
            patientid=path.split('/')[-1][:-4]
            #分别获取numpy数组的img和bboxes
            img = npy['image']
            bboxes = npy['bboxes']
            #进行仿射变换
            rot_img,rot_bboxes=rotate_img_bbox(img,bboxes,degree=-10)
            #进行平移变换
            shift_img,shift_bboxes=shift_pic_bboxes(img,bboxes)
            #保存train_png,格式为patiendid-rot和patientid-shift
            rot_path=os.path.join(png_train_dir,f'{patientid}-rot.png')
            cv2.imwrite(rot_path,rot_img)
            shift_path=os.path.join(png_train_dir,f'{patientid}-shift.png')
            cv2.imwrite(shift_path,shift_img)
            #保存新增的图像的class
            writer_c.writerow([patientid+'-rot','Lung Opacity'])
            writer_c.writerow([patientid+'-shift','Lung Opacity'])
            #保存新增图像的bbox,注意npy文件中的bbox为[y,x,y+height,x+width]
            for i in range(len(bboxes)):
                bbox=rot_bboxes[i]
                y,x,h,w=bbox
                h=h-y
                w=w-x
                writer_l.writerow([patientid+'-rot',int(x),int(y),int(w),int(h),1])
                bbox=shift_bboxes[i]
                y,x,h,w=bbox
                h=h-y
                w=w-x
                writer_l.writerow([patientid+'-shift',int(x),int(y),int(w),int(h),1])
            #保存变换后的bbox信息
            save_path = os.path.join(train_npy,f'{patientid}-rot.npy')
            np.save(save_path,{'image':rot_img,'bboxes':rot_bboxes})
            save_path = os.path.join(train_npy,f'{patientid}-shift.npy')
            np.save(save_path,{'image':shift_img,'bboxes':shift_bboxes})

  0%|          | 0/5659 [00:00<?, ?it/s]

  0%|          | 2/5659 [00:00<05:25, 17.36it/s]

  0%|          | 4/5659 [00:00<06:11, 15.22it/s]

  0%|          | 5/5659 [00:00<06:52, 13.69it/s]

  0%|          | 7/5659 [00:00<06:53, 13.65it/s]

...

100%|█████████▉| 5648/5659 [04:40<00:00, 20.13it/s]

100%|█████████▉| 5651/5659 [04:40<00:00, 20.13it/s]

100%|█████████▉| 5654/5659 [04:40<00:00, 20.13it/s]

100%|█████████▉| 5657/5659 [04:41<00:00, 20.13it/s]

100%|██████████| 5659/5659 [04:41<00:00, 20.13it/s]

In [18]:
# load and shuffle filenames
filenames = glob.glob('./input/stage_1_train_png/*.png')
#乱序filename
random.shuffle(filenames)
# split into train and validation filenames，设置验证集和训练集
n_valid_samples = 3400
train_filenames = filenames[n_valid_samples:]
valid_filenames = filenames[:n_valid_samples]
print('n train samples', len(train_filenames))
print('n valid samples', len(valid_filenames))
n_train_samples = len(filenames) - n_valid_samples
a=np.load('./input/stage_1_train_npy/4cbb68e5-1ca0-4a2a-b3d2-022110364d7c-rot.npy').item()
img=a['image']
print(img)
print(img.shape)
img=cv2.imread('./input/stage_1_train_png/4cbb68e5-1ca0-4a2a-b3d2-022110364d7c-rot.png')
print(img.shape)
img = pydicom.dcmread('./input/stage_1_train_images/4cbb68e5-1ca0-4a2a-b3d2-022110364d7c.dcm').pixel_array
print(img.shape)
dcm=sitk.ReadImage('./input/stage_1_train_images/4cbb68e5-1ca0-4a2a-b3d2-022110364d7c.dcm')
#print(dcm)
img_array=sitk.GetArrayFromImage(dcm)[0]
print(img_array)

n train samples 33602
n valid samples 3400
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(1024, 1024)
(1024, 1024, 3)
(1024, 1024)
[[  1   1   1 ...   4   3   2]
 [  1   1   1 ...   5   4   3]
 [  1   1   1 ...   5   5   3]
 ...
 [  1   1   1 ... 159 157  72]
 [  1   1   1 ... 161 163  88]
 [  1   1   1 ...  89 101  40]]


In [28]:
npy_path=glob.glob(train_npy+'*.npy')
print(len(npy_path))
pneumonia_locations={}
for i in range(len(npy_path)):
    npy = np.load(npy_path[i]).item()
    patientid=npy_path[i].split('/')[-1][:-4]
    bboxes = npy['bboxes']
    pneumonia_locations[patientid]=bboxes

16977


In [ ]:
#创建存储train——npy内容的字典
npy_dict={}
for path in tqdm(glob.glob(train_npy+'*.npy',recursive=True)):
    npy = np.load(path).item()
    patientid=path.split('/')[-1][:-4]
    #分别获取numpy数组的img和bboxes
    img = npy['image']
    bboxes = npy['bboxes']
    npy_dict[patientid]=[]
    npy_dict[patientid].append(img)
    npy_dict[patientid].append(bboxes)



In [40]:
print('Total train images:',len(filenames))
#患有肺炎的病人的个数，一个病人可能有多个opacities
print('patient with pneumonia:', len(pneumonia_locations))

ns = [len(value) for value in pneumonia_locations.values()]
plt.figure()
plt.hist(ns)
plt.xlabel('Pneumonia per image')
plt.xticks(range(1, np.max(ns)+1))
plt.show()

heatmap = np.zeros((1024, 1024))
ws = []
hs = []
for values in pneumonia_locations.values():
    for value in values:
        y, x, h, w = value
        y=int(y)
        x=int(x)
        h=int(h)
        w=int(w)
        heatmap[y:h, x:w] += 1
        ws.append(w-x)
        hs.append(h-y)
plt.figure()
plt.title('Pneumonia location heatmap')
plt.imshow(heatmap)
plt.figure()
plt.title('Pneumonia height lengths')
plt.hist(hs, bins=np.linspace(0,1000,50))
plt.show()
plt.figure()
plt.title('Pneumonia width lengths')
plt.hist(ws, bins=np.linspace(0,1000,50))
plt.show()
print('Minimum pneumonia height:', np.min(hs))
print('Minimum pneumonia width: ', np.min(ws))


Total train images: 37002
patient with pneumonia: 16977


Minimum pneumonia height: 45
Minimum pneumonia width:  40


In [47]:
class generator(keras.utils.Sequence):
    
    def __init__(self, folder, filenames, pneumonia_locations=None, batch_size=32, image_size=256, shuffle=True, augment=False, predict=False):
        self.folder = folder
        self.filenames = filenames
        self.pneumonia_locations = pneumonia_locations
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.augment = augment
        self.predict = predict
        self.on_epoch_end()
        self.train_npy='./input/stage_1_train_npy/'
        #self.npy_path=glob.glob(self.train_npy+'*.npy')
        
    def __load__(self, filename):
        # 通过filename确定patientid，然后读取npy文件获取img
        img_path=filename.split('.')[0]+'.npy'
        npy=np.load(os.path.join(self.train_npy,img_path)).item()
        img = npy['image']
        # create empty mask
        msk = np.zeros(img.shape)
        # get filename without extension
        filename = filename.split('.')[0]
        # if image contains pneumonia
        if filename in self.pneumonia_locations:
            # loop through pneumonia
            for location in self.pneumonia_locations[filename]:
                # add 1's at the location of the pneumonia
                #因为做过变换所以可能不是int类型
                y, x, h, w = location
                y=int(y)
                x=int(x)
                h=int(h)
                w=int(w)
                msk[y:h, x:w] = 1
        # resize both image and mask
        img = resize(img, (self.image_size, self.image_size), mode='reflect')
        msk = resize(msk, (self.image_size, self.image_size), mode='reflect') > 0.5
        # if augment then horizontal flip half the time
        if self.augment and random.random() > 0.5:
            img = np.fliplr(img)
            msk = np.fliplr(msk)
        # add trailing channel dimension
        img = np.expand_dims(img, -1)
        msk = np.expand_dims(msk, -1)
        return img, msk
    
    def __loadpredict__(self, filename):
        # 通过filename确定patientid，然后读取npy文件获取img
        img_path=filename.split('.')[0]+'.npy'
        npy=np.load(os.path.join(self.train_npy,img_path)).item()
        img = npy['image']
        img = resize(img, (self.image_size, self.image_size), mode='reflect')
        # add trailing channel dimension
        img = np.expand_dims(img, -1)
        return img
        
    def __getitem__(self, index):
        # select batch
        filenames = self.filenames[index*self.batch_size:(index+1)*self.batch_size]
        # predict mode: return images and filenames
        if self.predict:
            # load files
            imgs = [self.__loadpredict__(filename) for filename in filenames]
            # create numpy batch
            imgs = np.array(imgs)
            return imgs, filenames
        # train mode: return images and masks
        else:
            # load files
            items = [self.__load__(filename) for filename in filenames]
            # unzip images and masks
            imgs, msks = zip(*items)
            # create numpy batch
            imgs = np.array(imgs)
            msks = np.array(msks)
            return imgs, msks
        
    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.filenames)
        
    def __len__(self):
        if self.predict:
            # return everything
            return int(np.ceil(len(self.filenames) / self.batch_size))
        else:
            # return full batches only
            return int(len(self.filenames) / self.batch_size)

In [48]:
def create_downsample(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 1, padding='same', use_bias=False)(x)
    x = keras.layers.MaxPool2D(2)(x)
    return x

def create_resblock(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    return keras.layers.add([x, inputs])
#基本结构为残差网络
def create_network(input_size, channels, n_blocks=2, depth=4):
    # input
    inputs = keras.Input(shape=(input_size, input_size, 1))
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(inputs)
    # residual blocks
    for d in range(depth):
        channels = channels * 2
        x = create_downsample(channels, x)
        for b in range(n_blocks):
            x = create_resblock(channels, x)
    # output
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(1, 1, activation='sigmoid')(x)
    outputs = keras.layers.UpSampling2D(2**depth)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
# define iou or jaccard loss function
def iou_loss(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    #label被看作是1的mask，相乘可确定公共区域，reduce_sum返回该矩阵所有元素之和
    intersection = tf.reduce_sum(y_true * y_pred)
    score = (intersection + 1.) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection + 1.)
    return 1 - score

# combine bce loss and iou loss
def iou_bce_loss(y_true, y_pred):
    return 0.5 * keras.losses.binary_crossentropy(y_true, y_pred) + 0.5 * iou_loss(y_true, y_pred)

# mean iou as a metric
def mean_iou(y_true, y_pred):
    y_pred = tf.round(y_pred)
    intersect = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.reduce_sum(y_true, axis=[1, 2, 3]) + tf.reduce_sum(y_pred, axis=[1, 2, 3])
    smooth = tf.ones(tf.shape(intersect))
    #如果不设置axis，所有维度上的元素都会被求平均值，并且只会返回一个只有一个元素的张量。
    return tf.reduce_mean((intersect + smooth) / (union - intersect + smooth))

# create network and compiler
model = create_network(input_size=256, channels=32, n_blocks=2, depth=4)
model.compile(optimizer='adam',
              loss=iou_bce_loss,
              metrics=['accuracy', mean_iou])

# cosine learning rate annealing
def cosine_annealing(x):
    lr = 0.001
    epochs = 25
    return lr*(np.cos(np.pi*x/epochs)+1.)/2
learning_rate = tf.keras.callbacks.LearningRateScheduler(cosine_annealing)

# create train and validation generators
folder = './input/stage_1_train_png'
train_gen = generator(folder, train_filenames, pneumonia_locations, batch_size=32, image_size=256, shuffle=True, augment=True, predict=False)
valid_gen = generator(folder, valid_filenames, pneumonia_locations, batch_size=32, image_size=256, shuffle=False, predict=False)

history = model.fit_generator(train_gen, validation_data=valid_gen, callbacks=[learning_rate], epochs=25, workers=4, use_multiprocessing=True)

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(131)
plt.plot(history.epoch, history.history["loss"], label="Train loss")
plt.plot(history.epoch, history.history["val_loss"], label="Valid loss")
plt.legend()
plt.subplot(132)
plt.plot(history.epoch, history.history["acc"], label="Train accuracy")
plt.plot(history.epoch, history.history["val_acc"], label="Valid accuracy")
plt.legend()
plt.subplot(133)
plt.plot(history.epoch, history.history["mean_iou"], label="Train iou")
plt.plot(history.epoch, history.history["val_mean_iou"], label="Valid iou")
plt.legend()
plt.show()

In [ ]:
for imgs, msks in valid_gen:
    # predict batch of images
    preds = model.predict(imgs)
    # create figure
    f, axarr = plt.subplots(4, 8, figsize=(20,15))
    axarr = axarr.ravel()
    axidx = 0
    # loop through batch,每次循环把索引相同的值组成一个list
    for img, msk, pred in zip(imgs, msks, preds):
        # plot image
        axarr[axidx].imshow(img[:, :, 0])
        # threshold true mask
        comp = msk[:, :, 0] > 0.5
        # apply connected components
        comp = measure.label(comp)
        # apply bounding boxes
        predictionString = ''
        for region in measure.regionprops(comp):
            # retrieve x, y, height and width
            y, x, y2, x2 = region.bbox
            height = y2 - y
            width = x2 - x
            axarr[axidx].add_patch(patches.Rectangle((x,y),width,height,linewidth=2,edgecolor='b',facecolor='none'))
        # threshold predicted mask
        comp = pred[:, :, 0] > 0.5
        # apply connected components
        comp = measure.label(comp)
        # apply bounding boxes
        predictionString = ''
        for region in measure.regionprops(comp):
            # retrieve x, y, height and width
            y, x, y2, x2 = region.bbox
            height = y2 - y
            width = x2 - x
            axarr[axidx].add_patch(patches.Rectangle((x,y),width,height,linewidth=2,edgecolor='r',facecolor='none'))
        axidx += 1
    plt.show()
    # only plot one batch
    break

In [ ]:
# load and shuffle filenames
folder = './input/stage_1_test_png'
#读取指定目录下所有的文件名
test_filenames = os.listdir(folder)
print('n test samples:', len(test_filenames))

# create test generator with predict flag set to True
test_gen = generator(folder, test_filenames, None, batch_size=25, image_size=256, shuffle=False, predict=True)

# create submission dictionary
submission_dict = {}
# loop through testset
for imgs, filenames in test_gen:
    # predict batch of images
    preds = model.predict(imgs)
    # loop through batch
    for pred, filename in zip(preds, filenames):
        # resize predicted mask
        pred = resize(pred, (1024, 1024), mode='reflect')
        # threshold predicted mask,mask中大于0.5才为真
        comp = pred[:, :, 0] > 0.5
        # apply connected components
        comp = measure.label(comp)
        # apply bounding boxes
        predictionString = ''
        for region in measure.regionprops(comp):
            # retrieve x, y, height and width
            y, x, y2, x2 = region.bbox
            height = y2 - y
            width = x2 - x
            # proxy for confidence score
            conf = np.mean(pred[y:y+height, x:x+width])
            # add to predictionString
            predictionString += str(conf) + ' ' + str(x) + ' ' + str(y) + ' ' + str(width) + ' ' + str(height) + ' '
        # add filename and predictionString to dictionary
        filename = filename.split('.')[0]
        submission_dict[filename] = predictionString
    # stop if we've got them all
    if len(submission_dict) >= len(test_filenames):
        break

# save dictionary as csv file
sub = pd.DataFrame.from_dict(submission_dict,orient='index')
sub.index.names = ['patientId']
sub.columns = ['PredictionString']
sub.to_csv('/input/submission.csv')

In [27]:
out_image,rot_bboxes=rotate_img_bbox(img,bboxes,degree=-10)
def visulize(img,bboxes):
    copy_image = img.copy()
    for bbox in  bboxes:
        cv2.rectangle(copy_image,(int(bbox[1]),int(bbox[0])),(int(bbox[3]),int(bbox[2])),[0,255,0],4)
    plt.imshow(copy_image,'gray')
plt.figure(figsize=(20,20))
plt.subplot(121)
visulize(img,bboxes)
plt.subplot(122)
visulize(out_image,rot_bboxes)#如果只是变化左上角和右下角坐标的会出现这种情况，如果旋转角度小，到影响不道，45度影响最大，90度反而一样了 
plt.show()
cv2.imwrite('./input/1.png',out_image)

True

In [19]:
shift_img,shift_bboxes=shift_pic_bboxes(img,bboxes)
plt.figure(figsize=(20,20))
plt.subplot(121)
visulize(img,bboxes)
plt.subplot(122)
visulize(shift_img,shift_bboxes)#如果只是变化左上角和右下角坐标的会出现这种情况，如果旋转角度小，到影响不道，45度影响最大，90度反而一样了 
plt.show()

In [7]:

# pnemonia_locations存放对应病人id和bbox的位置
pneumonia_locations={}
# load table
with open(os.path.join('./input/stage_1_train_labels.csv'), mode='r') as infile:
    # open reader
    reader = csv.reader(infile)
    # skip header
    next(reader, None)
    # loop through rows
    for rows in reader:
        # retrieve information
        patientid = rows[0]
        location = rows[1:5]
        pneumonia = rows[5]
        if pneumonia == '1':
            location = [int(float(i)) for i in location]
            if patientid in pneumonia_locations:
                pneumonia_locations[patientid].append(location)
            else:
                pneumonia_locations[patientid] = [location]
#save a list of the patientid with lung opacities
pneumonia_id=[k for k in pneumonia_locations.keys()]
'''
img = pydicom.dcmread(os.path.join(dcm_train_dir, pneumonia_id[3]+'.dcm')).pixel_arrayay
print(img.shape)
#原来dcm图像的shape为1024*1024
plt.figure(figsize=(20,20))
plt.imshow(img)
plt.show()
'''
print(len(pneumonia_locations.keys()))

5659


In [84]:
bboxes=[]
for i in range(len(pneumonia_locations[pneumonia_id[3]])):
    x,y,w,h=pneumonia_locations[pneumonia_id[3]][i]
    #以坐标形式存储一个bbox的信息
    bboxes.append([y,x,y+h,x+w])
bboxes

[[544, 306, 788, 474], [511, 650, 795, 856]]

In [67]:
#创建存储train——npy内容的字典
npy_dict={}
for path in tqdm(glob.glob(train_npy+'*.npy',recursive=True)):
    npy = np.load(path).item()
    patientid=path.split('/')[-1][:-4]
    #分别获取numpy数组的img和bboxes
    img = npy['image']
    bboxes = npy['bboxes']
    npy_dict[patientid]=[]
    npy_dict[patientid].append(img)
    npy_dict[patientid].append(bboxes)

[[544, 306, 788, 474], [511, 650, 795, 856]]

In [69]:
dcm=sitk.ReadImage('./input/stage_1_train_png/344967.png')
img_array=sitk.GetArrayFromImage(dcm)
print(img_array.shape)
save_path = os.path.join(png_train_dir,'test.png')
cv2.imwrite(save_path,img_array)
img=cv2.imread(save_path)
img.shape


(1200, 1920, 4)


(1200, 1920, 3)

In [65]:
img = pydicom.dcmread('./input/stage_1_train_images/4cbb68e5-1ca0-4a2a-b3d2-022110364d7c.dcm').pixel_array
print(img.shape)
print('------')
cv2.imwrite('./input/test/1.png',img)
dcm=sitk.ReadImage('./input/stage_1_train_images/4cbb68e5-1ca0-4a2a-b3d2-022110364d7c.dcm')
img_array=sitk.GetArrayFromImage(dcm)[0]
print(img_array)


(1024, 1024)
------
[[  1   1   1 ...   4   3   2]
 [  1   1   1 ...   5   4   3]
 [  1   1   1 ...   5   5   3]
 ...
 [  1   1   1 ... 159 157  72]
 [  1   1   1 ... 161 163  88]
 [  1   1   1 ...  89 101  40]]
